**This notebook contains a script that fixes the moderated poes post assessment documentation**

**Add Local Library To Path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [2]:
import pandas as pd
import re
from local_library import list_files
from local_library import import_worksheet
from local_library import export_worksheet

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

#authenticate Google
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)


def move_file(old_folder_id, new_folder_id, exclusion_array=[]):
    """
    Moves a google drive files from one folder to another.
    
    Args:
        old_folder_id: A string representing the old folder google drive id.
        new_folder_id: A string representing the new folder google drive id.
        exclusion_array: An array of strings containing the titles to be excluded from the copy.
        
    Returns:
        _: A boolean representing success or failure
    """
    
    files = list_files(old_folder_id)
    for file in files:
        if file['title'] not in exclusion_array:
            copy_file(file['id'], file['title'], new_folder_id)
    return True
    
def copy_file(file_id, file_title, new_folder_id):
    """
    Creates copy of file in destination folder
    
    Args:
        file_id: I.D of the file to be copied
        file_title: Name of the file to be copied
        new_folder_id: Folder where copy will be stored.
    Return:
        none
    """
    
    copied_file = {"title": file_title,
                    'parents': [{
            'kind': 'drive#fileLink',
            'id': new_folder_id
        }]}

    drive.auth.service.files().copy(fileId=file_id,
                                    body=copied_file,
                                    supportsAllDrives=True
    ).execute()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Get Moderated POE Names**

In [20]:
#list files in moderated poes folder
files = list_files("12vNvTMr7wOYYrr6pG_juR13mNX_K7Q8B")

#get folder titles aka student names
student_names = []
for file in files:
    student_names.append([file['title']])
    
#convert folder titles to dataframe
moderated_df = pd.DataFrame(student_names, columns=["Name"])

**Import Data**

In [21]:
#import folders data
folders_df = import_worksheet("POE Folders Compilation","Folders")

#import documents id data
documents_df = import_worksheet("POE Folders Compilation","Cohort")
documents_df = documents_df[['Name','Evidence ID']]

**Merge DataFrames**

In [22]:
#merge moderated dataframe with documents id dataframe
temp_df = pd.merge(moderated_df, documents_df, how="left")

In [23]:
#merge temp dataframe with folders dataframe
cohort_df = pd.merge(temp_df, folders_df, how="left", left_on="Name", right_on="poe_folder_title")

In [24]:
cohort_df = cohort_df.drop('Name', 1)
cohort_df.head()

/var/folders/33/s663vbhn2v1f4q409xh14zkh0000gn/T/ipykernel_89340/2432240539.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cohort_df = cohort_df.drop('Name', 1)


,Evidence ID,poe_folder_title,poe_folder,documents_folder,presentation_folder,presentation_instruments_folder,presentation_evidence_folder,demonstration_folder,demonstration_instruments_folder,demonstration_evidence_folder,reports_folder,reports_instruments_folder,reports_evidence_folder,exams_folder,exams_formatives_folder,exams_summatives_folder
0,17eAtNmLYRW3ozsli2MNDEbom5LdPDVbK,Zanele Fortunate Manyathi,1WRZMyjRw6AzLBZ_PsirvjG_txmWyfMkv,1EQjBfySJU7vno8ItoOMIoWlp5al0YFgo,18LfRJyz9cu2K6Ab1Hil2ajNIhlf6T5SG,1O4wEDlSB7STrB5RcOJNwrnMbdY54QCDO,1gl0dl5k6DfLot-JJx2b7Az0StkR1fJ5K,1mwhRKCjfsCw_CpYNu_x-yzmejxovFZhp,1T7NwNkIVxASLeUajs3zjiT5MjMxIHkTT,1phznYu7uG7o87Zh-zgSpu2OogNrv4VNk,13mdmuch2GFkSWX9_4JgsqwFEUF9ijNe3,1S0_zrmR0Ab7nrIaegNxZDpDt5wrAfYWY,1PJ6o5dcvlgCiaDxdF5bKD9-b1hlj55bC,1MTzEQnY--kIRhOTUvbuybapH_7yBmqfi,1ldkmocBZcOL4DhpCoBKFtpd5FBA1VMSz,1vOc6908-sK3w2wcEc9Q3RYWfIcdPiEth
1,171Y-efSJp3Hz7gDExebnPdxIFbKiYQwt,Xolani Cyril Ndlangisa,1DkZulpR_86wCXsU0kvuBYUAL8LkfwbsQ,1syppq0w9VanIfW0FDouKFD6MOCxhKJq4,1gqIfQGkbERV5OT3_Nnfo3ER9rt4NHvFp,16lPOJ_p2dRnkdf_3yVPYLgFl0TySrWNu,1RliVPrZMCWynC6mq8n2xZ8YZQ56Tg9sw,155_RFNnIh4Wos6fNeHDzQhxrZkwMB9VM,1k-__-ea6Q_C6IeBm4dRE2xscicBqLvBr,1nMQSC0gVXqh2F9aGqalwlKud9ktrCxM4,1Lu9LuGVdWK-7yKeORwOIdQgYObbQLJGn,1vwfnmAyxhU7QK_Xy2gpcW0hT20envtwM,1jH5H8l2Lz6WcygRqRMsPeqW9NhnnXpvq,1JUOepaj0y2qxfb4ob_UxtGD_47PlQA1a,1ZAyGTcNKsnLbvtGt3H6GZQiFoDftdyVG,1R3JGosC9RbKB8J1wTT86A4Jdbx267-XZ
2,1v8S1mV4zQXv8hZKT41HqhMGG6BneJZMV,Wren Keanan Adams,1DQtNzNYbN3hLKKygU-LJDMC6sreJ_kTl,1WMx1xYbButJEqx4wcQXpBjgUklmFFiPl,14duFRf_h9AIo91gF51artX9aF7fGYbNS,1FUzxyvGC4jb10EekcMZ4ExAY-nVO4RUC,10YsvAVjT0m-Vk1L3TdqD-s-MsRiCN7JN,1HvFXtiNi1TWTev_rLoGn5BU9yTUQEQ2G,1LqV_c6Kz6qGiDIuKALd2OdkWSJkaq54M,1zPxO5UHBTgtdG54cp0C0bfbq7hweUY5k,1S6Elum6Le6vULB_Ggqyn535y_lThU79t,1oDjqa1pqcbc1nmr6mGvOZqMxFItdpnoS,10HlxyHLDKD52dg-sLIo4vMHNEIA3G6YC,1mPUV8urDy-7GEA1JYKK70dflYuWHAuZc,1OhbmcijwH6ZgPvlBx3UJNZyQaTCU6tfg,1twrmv0b8kmHFzI9V-bI6l6Jv_cjFRs8j
3,1F_KnqY6mIVU4Y5WMUkdv8Dd4ZZEmOYQ8,Wilson Kolana Makoae,1nsTg0iNvb3Dfi8XF6ImSnxpP4iTv4cl8,15EdRw_uQLJYDMDk8YgWD-LjGbSgSLitz,1qI_LIwxX2vav9-S8u1fC6F5iWGmI6cB_,1QbvUnd2nzNWxCjYQzk9UWaDZrEGgSQPi,1Ft1KGJuPOhTeDiOxpN-qgIvxIz80935W,162leFx_4g0K_nSAeNwBw7IHqy3LOsOS1,1TU3YfrPfXdEnBIIfGRORezzbXJJlNP2N,1OO11o060YVs7n0yLqJu7lNNTdaybZLPT,1vVdZIYiq9C9BlHWqG8uICYzfWow2xVVc,1KrGvLcipQJciZATOMDngtWsDyWQg4B81,1_cHtnnoyclVClPKlcTf0nYEMDaaSJ8a9,1pJUE4w2ZNtxwR3jPPTPrU-o-erwg2Y5N,1YxdpsgjyguVvdOWlThsZ_CSNqtVCd_SZ,1JHVG5Q2G7EYyOLXcJLQrKgenylTyu0qH
4,1ScVaDegIpLSTvlbA4otKQf0Ke7GIPnFp,Isaac Vusi Sithole,1DlnpMUt9mfGfs0RUpa5MohVmkSfyxDH5,1zM7ZPfpGwhnbymwIY7t7PdD-a8rnI_9t,1Vrjcr1Tp3Lor0CTfL4x_zpOe0hDVbWgZ,13PS01IsqnsCTlqpHIR3rsIaLEYUOex8s,1XzDIVlVezTVwz8R88bvz3C9KCbqYG3jp,1cKHbEpYtdmOxWSEmoC13YT92P2NCrzLH,1oPWbylMWQBKfg3Ab-roQkBY9v1GJG2T5,1myiIKgDfq2qt6AEWj1QRGyw-RdGoMPwa,1epAuljSTqvDqeVAP2XE3qEEP7FVOrfp4,1Y4d4gbVq9kGqFHOz2Xg_RVHF_4kiefGV,1al3OvMVrezr7M33i5V1dHes3mykN4bHq,1RycBYmuuT70vTb8sM2Ws-uFOfydUvOMB,1mkcCjTkUWa67jtlwEtFUdkAW29a6SN5U,1M9RYzCBEBbAhiuXeFuvUWDisSVnW9uzg


**Copy Post Assessment Documents**

In [27]:
for index, row in cohort_df[cohort_df['fixed'] != 'Yes'].iterrows():
    #log start
    print(f"{row['poe_folder_title']}: Start Copy")

    #list all sub folders in evidence folder
    evidence_folders = list_files(row['Evidence ID'])

    #move all presentation post assessments docs into presentation folder
    exclusion_array = ['demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], row['presentation_folder'], exclusion_array)
    
    #move all demonstration post assessments docs into demonstration folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], row['demonstration_folder'], exclusion_array)
    
    #move all reports post assessments docs into reports folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], row['reports_folder'], exclusion_array)
    
    #move all exams post assessments docs into exams folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], row['exams_folder'], exclusion_array)
    
    #log end
    cohort_df.loc[cohort_df['poe_folder_title'] == row['poe_folder_title'], 'fixed'] = 'Yes'
    print(f"{row['poe_folder_title']}: End Copy")
    

Wren Keanan Adams: Start Copy
Wren Keanan Adams: End Copy
Wilson Kolana Makoae: Start Copy
Wilson Kolana Makoae: End Copy
Isaac Vusi Sithole: Start Copy
Isaac Vusi Sithole: End Copy
Vusmuzi Gift Pekane: Start Copy
Vusmuzi Gift Pekane: End Copy
Tokollo Motlohi Ramabina: Start Copy
Tokollo Motlohi Ramabina: End Copy
Seokamela David Ntjobokoane: Start Copy
Seokamela David Ntjobokoane: End Copy
Siphesihle Manana: Start Copy
Siphesihle Manana: End Copy
Ntlopo Shadrack Mabitsela: Start Copy
Ntlopo Shadrack Mabitsela: End Copy
Rorisang Mmafoka Monatisa: Start Copy
Rorisang Mmafoka Monatisa: End Copy
Oarabile Sebotse Moima: Start Copy
Oarabile Sebotse Moima: End Copy
Ntshwara Princess Lamola: Start Copy
Ntshwara Princess Lamola: End Copy
Nhlanhla Themba: Start Copy
Nhlanhla Themba: End Copy
Nobantu Hope Gumbi: Start Copy
Nobantu Hope Gumbi: End Copy
Ntsikelelo Camagu Ngcai: Start Copy
Ntsikelelo Camagu Ngcai: End Copy
Nwabisa Ndengezi: Start Copy
Nwabisa Ndengezi: End Copy
Niyaaz Mohammed Moos